In [6]:
import numpy as np
import PIL
import PIL.Image
import torch
import json
import os
import xml.etree.ElementTree as ET

use_gpu = torch.cuda.is_available()
device = torch.device('cuda') if use_gpu else torch.device('cpu')

In [7]:
# calculate different path numbers stroke with GT and store json record
# 計算每一筆劃使用不同路徑時的MSE
gt_direct = "resource/flower/single_strokes//sunflowers_rendered_single_stroke_"
test_direct = "log/flower/path10/sunflowers_rendered_single_stroke_"
record = {}

for i in range(1, 496) :
    num = str(i)
    if len(num) == 1 :
        num = '000' + num
    if len(num) == 2:
        num = '00' + num
    if len(num) == 3:
        num = '0' + num

    gt_filname = gt_direct +  num + ".png"
    # example-> gt_filname = resource/flower/single_strokes//sunflowers_rendered_single_stroke_0001.png
    gt = np.array(PIL.Image.open(gt_filname))

    if gt.shape[2] == 4:
        #print("Input image includes alpha channel, simply dropout alpha channel.")
        gt = gt[:, :, :3]

    gt = (gt/255).astype(np.float32)
    gt = torch.FloatTensor(gt).permute(2, 0, 1)[None].to(device)

    loss_list = []

    for j in range(1, 11) :
        
        path_num = str(j)
        stroke_filname = test_direct + num + "/demo-png/" + path_num + ".png"
        # example -> stroke_filename = log/flower/path10/sunflowers_rendered_single_stroke_0001/demo-png/1.png
        pred = np.array(PIL.Image.open(stroke_filname))

        if pred.shape[2] == 4:
            #print("Input image includes alpha channel, simply dropout alpha channel.")
            pred = pred[:, :, :3]

        pred = (pred/255).astype(np.float32)
        pred = torch.FloatTensor(pred).permute(2, 0, 1)[None].to(device)
        
        loss = ((pred-gt)**2)
        loss = loss.sum(1).mean().item()
        #print(loss)
        loss_list.append(float(loss))
    record[i] = loss_list

results = {
    'loss': record,
}

#print(json.dumps(results,indent=4))

with open(f'./stroke_path_mse.json', 'w') as writer:
    json.dump(results, writer, indent=4)

In [8]:
#算全部筆畫的tolerence mse path num
f = open('./stroke_path_mse.json')
data = json.load(f)

data = data['loss']
list = []
for item in data :
    print(item)
    mse_data = np.array(data[item])
    mse_data = mse_data.astype(np.float32)

    # Set the initial MSE value and tolerance level
    initial_mse = (mse_data[0] - mse_data[1])
    tolerance = 0.1 * initial_mse  # Set the tolerance level to 1% of the initial MSE value

    # Check for convergence
    converged = False
    for i in range(1, len(mse_data)):
        if -1*(mse_data[i] - mse_data[i-1]) < tolerance:
            list.append(i)
            print(f'Converged after {i} epochs')
            print(mse_data[i])
            converged = True
            break

    if not converged:
        list.append(len(mse_data))
        print('Did not converge')

results = {
    'stroke num': list,
}


with open(f'./stroke_tolerance.json', 'w') as writer:
    json.dump(results, writer, indent=4)

1
Converged after 4 epochs
0.010947591
2
Converged after 3 epochs
0.008420533
3
Converged after 4 epochs
0.002985493
4
Converged after 5 epochs
0.003482895
5
Converged after 2 epochs
0.003406271
6
Converged after 2 epochs
0.002800473
7
Converged after 2 epochs
0.002125231
8
Converged after 3 epochs
0.0036882167
9
Converged after 5 epochs
0.0006915728
10
Converged after 3 epochs
0.00039888496
11
Converged after 3 epochs
0.001754515
12
Converged after 3 epochs
0.0017718583
13
Converged after 3 epochs
0.0008555595
14
Converged after 4 epochs
0.0015394727
15
Converged after 7 epochs
0.00022234664
16
Converged after 2 epochs
0.0012019079
17
Converged after 2 epochs
0.0003581708
18
Converged after 3 epochs
0.0002383366
19
Converged after 3 epochs
0.0003367315
20
Converged after 5 epochs
0.00029539055
21
Converged after 2 epochs
0.00019135434
22
Converged after 2 epochs
0.0013443199
23
Converged after 2 epochs
0.00016378194
24
Converged after 4 epochs
0.00018749168
25
Converged after 3 epochs

In [9]:
#根據tolerence stroke num path 合成結果
data = json.load(open('./stroke_tolerance.json'))
stroke_num_data = np.array(data['stroke num'])

label = json.load( open('./stroke_size_clustering_results.json') )
stroke_label_data = np.array(label['label'])

label_0 = 0
label_1 = 0 
label_2 = 0
label_0_size = 0
label_1_size = 0 
label_2_size = 0

for i in range(0,(stroke_num_data).size-1):
    if stroke_label_data[i] == 0 :
        label_0 += stroke_num_data[i]
        label_0_size += 1
    if stroke_label_data[i] == 1 :
        label_1 += stroke_num_data[i]
        label_1_size += 1
    if stroke_label_data[i] == 2 :
        label_2 += stroke_num_data[i]
        label_2_size += 1
        
label_0_averrage = round(label_0 / label_0_size)
label_1_averrage = round(label_1 / label_1_size)
label_2_averrage = round(label_2 / label_2_size)
print(label_0_averrage,label_1_averrage,label_2_averrage)


4 4 3


In [10]:
# former_animateID = "a0001"
# for i in range(0, 495) :
#     num = str(i+1)
#     if len(num) == 1 :
#         num = '000' + num
#     if len(num) == 2:
#         num = '00' + num
#     if len(num) == 3:
#         num = '0' + num
#     pathID = "stroke_" + num
#     animateID = "a" + num
#     path_num = str(4) #str(stroke_num_data[i]) # different path num
#     # if stroke_label_data[i] == 0 :
#     #     path_num = str(label_0_averrage)
#     # if stroke_label_data[i] == 1 :
#     #     path_num = str(label_1_averrage)
#     # if stroke_label_data[i] == 2 :
#     #     path_num = str(label_2_averrage)
#     # if stroke_label_data[i] == 3 :
#     #     path_num = str(label_3_averrage)
#     # if stroke_label_data[i] == 4 :
#     #     path_num = str(label_4_averrage)       
#     stroke_filname = file_direct + num + "/output-svg/" + path_num + ".svg"
#     #print(stroke_filname)
#     group = ET.Element('g')
#     group.set('id', pathID)
#     tree1 = ET.parse(stroke_filname)
#     root1 = tree1.getroot()
#     for path in root1.iter('{http://www.w3.org/2000/svg}path'):
#         fill_attrib = path.attrib.get('fill')
#         rgb_values = fill_attrib.split(',')
#         r = int(rgb_values[0][4:])
#         g = int(rgb_values[1])
#         b = int(rgb_values[2][:-1])
#         brightness = (r + g + b) / 3
#         if brightness < 240:
#             group.append(path)

#     animate1 = ET.Element('animate', {
#         'attributeType': 'XML',
#         'attributeName': 'opacity',
#         'from': '0',
#         'to': '0',
#         'begin': '0s',
#         'fill': 'freeze'
#     })
#     group.append(animate1)
#     animate2 = ET.Element('animate', {
#         'id': animateID,
#         'attributeType': 'XML',
#         'attributeName': 'opacity',
#         'from': '0',
#         'to': '1',
#         'dur': '50ms',
#         'begin': former_animateID + '.end+50ms',
#         'fill': 'freeze'
#     })
#     group.append(animate2)
#     svg.append(group)
#     former_animateID = animateID
# tree = ET.ElementTree(svg)   
# tree.write('output/test/merged_4.svg')
